# How to use AutoGluon for Kaggle competitions



In [1]:
# !pip install kaggle

In [2]:
!gdown 1seOAJxGY0MxdR90-emKqGFY0ejUcp8tn
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=1seOAJxGY0MxdR90-emKqGFY0ejUcp8tn
To: /content/kaggle.json
100% 69.0/69.0 [00:00<00:00, 393kB/s]


In [3]:
!kaggle competitions download -c ieee-fraud-detection

 97% 115M/118M [00:01<00:00, 80.8MB/s]
100% 118M/118M [00:01<00:00, 66.3MB/s]


In [4]:
!unzip ieee-fraud-detection.zip

Archive:  ieee-fraud-detection.zip
  inflating: sample_submission.csv   
  inflating: test_identity.csv       
  inflating: test_transaction.csv    
  inflating: train_identity.csv      
  inflating: train_transaction.csv   


In [6]:
!python -m pip install --upgrade pip --quiet
!python -m pip install autogluon.tabular[all] --quiet

In [7]:
!pip install --upgrade pyarrow --quiet

In [8]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

directory = '/content/' # directory where you have downloaded the data CSV files from the competition
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
save_path = directory + 'AutoGluonModels/'  # where to store trained models

train_identity = pd.read_csv(directory+'train_identity.csv')
train_transaction = pd.read_csv(directory+'train_transaction.csv')

In [9]:
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

In [10]:
# Refactor with a focus on resource efficiency
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=2).fit(
    train_data, presets='best_quality', time_limit=60, ag_args_fit={'num_gpus': 0, 'num_cpus': 2}, excluded_model_types=['KNN', 'XT']
)

results = predictor.fit_summary()

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       6.28 GB / 12.67 GB (49.6%)
Disk Space Avail:   61.61 GB / 112.64 GB (54.7%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacke

OutOfMemoryError: Task was killed due to the node running low on memory.
Memory on the node (IP: 172.28.0.12, ID: eceb84e7e9f5bc047eec5c950f38a65749a2b918d77f931dcad270aa) where the task (task ID: 1dd1256c3118aa82ba685170bc65a77f3b05f41701000000, name=_dystack, pid=2490, memory used=3.31GB) was running was 12.08GB / 12.67GB (0.953373), which exceeds the memory usage threshold of 0.95. Ray killed this worker (ID: cfbf731d287a7c822cbacfae159a2c98b000324754f227502664c443) because it was the most recently scheduled task; to see more information about memory usage on this node, use `ray logs raylet.out -ip 172.28.0.12`. To see the logs of the worker, use `ray logs worker-cfbf731d287a7c822cbacfae159a2c98b000324754f227502664c443*out -ip 172.28.0.12. Top 10 memory users:
PID	MEM(GB)	COMMAND
289	5.32	/usr/bin/python3 -m colab_kernel_launcher -f /root/.local/share/jupyter/runtime/kernel-e79207da-c0e9...
2490	3.31	ray::_dystack
2318	0.29	node /datalab/web/pyright/pyright-langserver.js --stdio --c

In [ ]:
test_identity = pd.read_csv(directory+'test_identity.csv')
test_transaction = pd.read_csv(directory+'test_transaction.csv')
test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')  # same join applied to training files

y_predproba = predictor.predict_proba(test_data)
y_predproba.head(5)  # some example predicted fraud-probabilities

In [ ]:
predictor.positive_class

In [ ]:
predictor.class_labels  # classes in this list correspond to columns of predict_proba() output

In [ ]:
y_predproba = predictor.predict_proba(test_data, as_multiclass=False)

In [ ]:
submission = pd.read_csv(directory+'sample_submission.csv')
submission['isFraud'] = y_predproba
submission.head()
submission.to_csv(directory+'my_submission.csv', index=False)